In [106]:
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [107]:
data = pd.read_csv('download.csv')#.transpose()

data = data.drop(0, axis=0)

In [108]:
class_data = pd.DataFrame(0, columns=['sta', 'sol', 'dyn', 'date'], index=[i for i in range(len(data.columns)-1)])

for i, row in enumerate(data.iloc()):
    name = row['Nth day']

    if 'sta' in name or "/course_instance/158188" in name:
        class_data['sta'] += np.array([int(i) for i in row[1:].values])

    if 'sol' in name or "/course_instance/157231" in name:
        class_data['sol'] += np.array([int(i) for i in row[1:].values])
    
    if 'dyn' in name or "/course_instance/158221" in name:
        class_data['dyn'] += np.array([int(i) for i in row[1:].values])

dates = [dt.datetime(year=2024, month=10, day=4) + dt.timedelta(days=i) for i in range(len(class_data.index))]

class_data['date'] = dates

class_data = class_data.set_index('date')

day = dt.datetime(year=2024, month=10, day=6)

days = [[] for i in range(7)]

for i in range(7):
    temp = day + dt.timedelta(days=i)
    while temp < dt.datetime(year=2024, month=11, day=17):
        days[i].append(temp)
        temp += dt.timedelta(days=7)

days

[[datetime.datetime(2024, 10, 6, 0, 0),
  datetime.datetime(2024, 10, 13, 0, 0),
  datetime.datetime(2024, 10, 20, 0, 0),
  datetime.datetime(2024, 10, 27, 0, 0),
  datetime.datetime(2024, 11, 3, 0, 0),
  datetime.datetime(2024, 11, 10, 0, 0)],
 [datetime.datetime(2024, 10, 7, 0, 0),
  datetime.datetime(2024, 10, 14, 0, 0),
  datetime.datetime(2024, 10, 21, 0, 0),
  datetime.datetime(2024, 10, 28, 0, 0),
  datetime.datetime(2024, 11, 4, 0, 0),
  datetime.datetime(2024, 11, 11, 0, 0)],
 [datetime.datetime(2024, 10, 8, 0, 0),
  datetime.datetime(2024, 10, 15, 0, 0),
  datetime.datetime(2024, 10, 22, 0, 0),
  datetime.datetime(2024, 10, 29, 0, 0),
  datetime.datetime(2024, 11, 5, 0, 0),
  datetime.datetime(2024, 11, 12, 0, 0)],
 [datetime.datetime(2024, 10, 9, 0, 0),
  datetime.datetime(2024, 10, 16, 0, 0),
  datetime.datetime(2024, 10, 23, 0, 0),
  datetime.datetime(2024, 10, 30, 0, 0),
  datetime.datetime(2024, 11, 6, 0, 0),
  datetime.datetime(2024, 11, 13, 0, 0)],
 [datetime.datetime(

In [114]:
daily_avgs_no_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7)])
daily_avgs_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7)])
daily_no_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7*3)])
daily_quiz = pd.DataFrame(0, columns=['sta', 'sol', 'dyn'], index=[i+1 for i in range(7*3)])

for i in range(7):
    for j, day in enumerate(days[i]):
        if not j % 2:
            daily_quiz.iloc[i+(j//2)*7]['sol'] = np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_no_quiz.iloc[i+(j//2)*7]['dyn'] = np.array(class_data.loc[class_data.index == day].values[0][2])
            daily_avgs_quiz.iloc[i]['sol'] += np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_avgs_no_quiz.iloc[i]['dyn'] += np.array(class_data.loc[class_data.index == day].values[0][2])
        else:
            daily_no_quiz.iloc[i+(j//2)*7]['sol'] = np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_quiz.iloc[i+(j//2)*7]['dyn'] = np.array(class_data.loc[class_data.index == day].values[0][2])
            daily_avgs_no_quiz.iloc[i]['sol'] += np.array(class_data.loc[class_data.index == day].values[0][1])
            daily_avgs_quiz.iloc[i]['dyn'] += np.array(class_data.loc[class_data.index == day].values[0][2])
    
    for j, day in enumerate(days[i]):

        if j in [0, 2, 5]:
            daily_quiz.iloc[i+(j//2)*7]['sta'] = np.array(class_data.loc[class_data.index == day].values[0][0])
            daily_avgs_quiz.iloc[i]['sta'] += np.array(class_data.loc[class_data.index == day].values[0][0])
        else:
            daily_no_quiz.iloc[i+(j//2)*7]['sta'] = np.array(class_data.loc[class_data.index == day].values[0][0])
            daily_avgs_no_quiz.iloc[i]['sta'] += np.array(class_data.loc[class_data.index == day].values[0][0])

for i, class_ in enumerate(['sta', 'sol', 'dyn']):
    r = pd.DataFrame(index=[i+1 for i in range(6)], columns=['quiz', 'noquiz'])
    r['quiz'] = daily_avgs_quiz[class_]
    r['noquiz'] = daily_avgs_no_quiz[class_]

    r.to_csv(f"{class_}.csv")

In [121]:
for class_ in ['sta', 'sol', 'dyn']:
    fig = px.line(class_data, x=class_data.index, y=class_)
    for week in range(7):
        init = dt.datetime(year=2024, month=10, day=6)
        if class_ == "sol" and week in [0, 2, 4]:
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=3.5), line_width=3, line_dash="dash", line_color="red")
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=6.5), line_width=3, line_dash="dash", line_color="red")
        if class_ == 'sol' and week == 6:
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=2.5), line_width=3, line_dash="dash", line_color="blue")
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=5.5), line_width=3, line_dash="dash", line_color="blue")
        if class_ == "sta" and week in [0, 2, 5]:
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=2.5), line_width=3, line_dash="dash", line_color="red")
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=5.5), line_width=3, line_dash="dash", line_color="red")
        if class_ == "dyn" and week in [1, 3, 5]:
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=1.5), line_width=3, line_dash="dash", line_color="red")
            fig.add_vline(x=init+dt.timedelta(weeks=week, days=4.5), line_width=3, line_dash="dash", line_color="red")
    fancy_name = "Dynamics" if class_ == 'dyn' else ('Solid Mechanics' if class_ == 'sol' else 'Statics')
    fig.update_layout(
        title=fancy_name,
        xaxis=dict(title="Date"),
        yaxis=dict(title="Total visitorship")
    )
    fig.show()

In [116]:
for class_ in ['sta', 'sol', 'dyn']:
    graph_data = pd.read_csv(f'{class_}.csv')
    graph_data = graph_data.set_index(graph_data.columns[0])
    fig = px.bar(graph_data, x=graph_data.index, y=["quiz", "noquiz"], barmode='group')
    if class_ == "sol":
        fig.add_vline(x=4.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=7.5, line_width=3, line_dash="dash", line_color="red")
    if class_ == "sta":
        fig.add_vline(x=3.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=6.5, line_width=3, line_dash="dash", line_color="red")
    if class_ == "dyn":
        fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="red")
        fig.add_vline(x=5.5, line_width=3, line_dash="dash", line_color="red")
    fancy_name = "Dynamics" if class_ == 'dyn' else ('Solid Mechanics' if class_ == 'sol' else 'Statics')
    fig.update_layout(
        title=fancy_name,
        xaxis=dict(title="Day of week (1 = sunday)"),
        yaxis=dict(title="Total visitorship")
    )
    fig.show()

In [117]:
import scipy.stats as stats
 
for i, class_ in enumerate(['sta', 'dyn', 'sol']):
    fancy_name = "Dynamics" if class_ == 'dyn' else ('Solid Mechanics' if class_ == 'sol' else 'Statics')
    quiz = daily_quiz[class_].values
    noquiz = daily_no_quiz[class_].values

    t, p = stats.ttest_ind(quiz, noquiz, alternative='greater')
    print(f"{fancy_name} & {round(t, 3)} & {round(p, 5)} \\\\")

Statics & 2.409 & 0.01034 \\
Dynamics & 2.532 & 0.00768 \\
Solid Mechanics & 1.699 & 0.04853 \\
